# PDF Note Space Adder

This notebook adds extra space on the right side of PDF pages with an optional grid pattern.

In [26]:
# Install required packages (only needed for the first time)
# !pip install pdf2image pillow img2pdf ipywidgets

In [27]:
import sys
import io
from pathlib import Path
from pdf2image import convert_from_path
from PIL import Image, ImageDraw
import img2pdf
from ipywidgets import FileUpload, IntSlider, VBox, Button, Label, Output, HBox
from IPython.display import display, clear_output


In [28]:
def create_grid_pattern(width, height, grid_size=20, color=(200, 200, 200)):
    """
    Create a grid pattern image.
    
    Args:
        width: Width of the grid image
        height: Height of the grid image
        grid_size: Size of each grid cell in pixels
        color: RGB color of the grid lines
        
    Returns:
        PIL Image with grid pattern
    """
    # Create a white image
    img = Image.new('RGB', (width, height), 'white')
    draw = ImageDraw.Draw(img)
    
    # Draw vertical lines
    x = 0
    while x <= width:
        draw.line([(x, 0), (x, height)], fill=color, width=1)
        x += grid_size
    
    # Draw horizontal lines
    y = 0
    while y <= height:
        draw.line([(0, y), (width, y)], fill=color, width=1)
        y += grid_size
    
    return img


In [29]:
def add_note_space(input_pdf_path, output_pdf_path, space_width_percent=50, grid_size=20, grid_color=(200, 200, 200)):
    """
    Add note space on the right side of PDF pages with optional grid pattern.
    
    Args:
        input_pdf_path: Path to input PDF file
        output_pdf_path: Path to output PDF file
        space_width_percent: Width of note space as percentage of original page width (default: 50%)
        grid_size: Size of grid cells in pixels (default: 20). Set to 0 to disable grid.
        grid_color: RGB color of grid lines (default: light gray)
    """
    input_path = Path(input_pdf_path)
    output_path = Path(output_pdf_path)
    
    if not input_path.exists():
        raise FileNotFoundError(f"Input PDF not found: {input_path}")
    
    # Convert PDF pages to images
    try:
        pages = convert_from_path(str(input_path), dpi=200)
    except Exception as e:
        raise Exception(f"Error converting PDF: {e}\nMake sure you have poppler installed:\n  macOS: brew install poppler\n  Linux: sudo apt-get install poppler-utils\n  Windows: Download from https://github.com/oschwartz10612/poppler-windows/releases")
    
    # Process each page
    processed_pages = []
    for page in pages:
        # Get original dimensions
        orig_width, orig_height = page.size
        
        # Calculate note space width
        note_space_width = int(orig_width * space_width_percent / 100)
        new_width = orig_width + note_space_width
        
        # Create new image with extra space
        new_image = Image.new('RGB', (new_width, orig_height), 'white')
        
        # Paste original page on the left
        new_image.paste(page, (0, 0))
        
        # Create and paste grid pattern on the right side
        if grid_size > 0:
            grid_pattern = create_grid_pattern(note_space_width, orig_height, grid_size, grid_color)
            new_image.paste(grid_pattern, (orig_width, 0))
        
        processed_pages.append(new_image)
    
    # Convert images back to PDF
    from io import BytesIO
    
    # Convert PIL Images to BytesIO objects for img2pdf
    image_bytes_list = []
    for img in processed_pages:
        img_bytes = BytesIO()
        img.save(img_bytes, format='PNG')
        img_bytes.seek(0)
        image_bytes_list.append(img_bytes)
    
    with open(output_path, 'wb') as f:
        pdf_bytes = img2pdf.convert(image_bytes_list)
        f.write(pdf_bytes)


## Usage

Use the interactive controls below to select your PDF file and adjust settings, then click "Process PDF" to add note space.


In [ ]:
# Create interactive widgets
file_upload = FileUpload(
    accept='.pdf',
    multiple=False,
    description='Select PDF:',
    style={'description_width': 'initial'}
)

space_width_slider = IntSlider(
    value=50,
    min=10,
    max=150,
    step=10,
    description='Note Space Width (%):',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

grid_size_slider = IntSlider(
    value=20,
    min=0,
    max=80,
    step=5,
    description='Grid Size (px, 0=no grid):',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

process_button = Button(
    description='Process PDF',
    button_style='success',
    icon='check',
    layout={'width': '200px'}
)

output_area = Output()

def get_uploaded_file_info(upload_value):
    """Extract filename and content from FileUpload value (handles both dict and tuple formats)"""
    if not upload_value:
        return None, None
    
    # Handle tuple format with Bunch objects: ({'name': 'file.pdf', 'content': <memory>, ...},)
    if isinstance(upload_value, tuple) and len(upload_value) > 0:
        file_info = upload_value[0]
        
        # Bunch objects can be accessed like dicts
        filename = None
        file_content = None
        
        # Try to get filename
        if hasattr(file_info, 'name'):
            filename = file_info.name
        elif isinstance(file_info, dict) and 'name' in file_info:
            filename = file_info['name']
        elif hasattr(file_info, 'get'):
            filename = file_info.get('name')
        
        # Try to get content
        if hasattr(file_info, 'content'):
            content_obj = file_info.content
        elif isinstance(file_info, dict) and 'content' in file_info:
            content_obj = file_info['content']
        elif hasattr(file_info, 'get'):
            content_obj = file_info.get('content')
        else:
            content_obj = None
        
        # Convert content to bytes if needed
        if content_obj is not None:
            if isinstance(content_obj, bytes):
                file_content = content_obj
            elif isinstance(content_obj, memoryview):
                file_content = content_obj.tobytes()
            elif hasattr(content_obj, 'tobytes'):
                file_content = content_obj.tobytes()
            elif hasattr(content_obj, 'read'):
                file_content = content_obj.read()
            else:
                # Try to convert to bytes
                try:
                    file_content = bytes(content_obj)
                except:
                    file_content = None
        
        if filename and file_content:
            return filename, file_content
    
    # Handle dict format: {filename: {'content': bytes, 'metadata': {...}}}
    elif isinstance(upload_value, dict):
        if len(upload_value) > 0:
            filename = list(upload_value.keys())[0]
            file_data = upload_value[filename]
            if isinstance(file_data, dict):
                if 'content' in file_data:
                    content_obj = file_data['content']
                    if isinstance(content_obj, bytes):
                        return filename, content_obj
                    elif isinstance(content_obj, memoryview):
                        return filename, content_obj.tobytes()
                else:
                    # Try to find bytes in the dict
                    for key, value in file_data.items():
                        if isinstance(value, bytes):
                            return filename, value
                        elif isinstance(value, memoryview):
                            return filename, value.tobytes()
            elif isinstance(file_data, bytes):
                return filename, file_data
            elif isinstance(file_data, memoryview):
                return filename, file_data.tobytes()
    
    return None, None

def on_file_upload(change):
    """Callback when file is uploaded"""
    pass

def on_process_button_clicked(b):
    """Process the uploaded PDF"""
    with output_area:
        clear_output()
        
        # Check if file is uploaded
        if not file_upload.value:
            print("❌ Please select a PDF file first!")
            return
        
        try:
            # Get filename and file content
            filename, file_content = get_uploaded_file_info(file_upload.value)
            
            if filename is None or file_content is None:
                print("❌ Could not read uploaded file. Please try uploading again.")
                return
            
            # Save to temporary file
            temp_input = Path("temp_input.pdf")
            with open(temp_input, 'wb') as f:
                f.write(file_content)
            
            # Generate output filename
            output_filename = filename.replace('.pdf', '_with_notes.pdf')
            temp_output = Path(output_filename)
            
            # Process the PDF
            add_note_space(
                str(temp_input),
                str(temp_output),
                space_width_slider.value,
                grid_size_slider.value
            )
            
            # Clean up temp file
            temp_input.unlink()
            
            print(f"✅ Done! Download your file: {output_filename}")
            
        except Exception as e:
            print(f"❌ Error: {e}")

file_upload.observe(on_file_upload, names='value')
process_button.on_click(on_process_button_clicked)

# Display the interface
display(VBox([
    Label("📄 Select PDF File:"),
    file_upload,
    Label("⚙️ Settings:"),
    space_width_slider,
    grid_size_slider,
    process_button,
    output_area
]))